<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

제일먼저 구글드라이버에 nmt-fold에 code1과 data1이 설치되도록 자신의 구글드라이버에 제공되는 zip 파일을 압축 해제해서 upload 하세요. 그리고 좌측 mount 버튼을 누르세요. 그러면 구글 드라이브 마운트 위치 "/content/drive/MyDrive/"에 마운트가 되어서 /content/drive/MyDrive/nmt_fold/data1/ 형태로 접근이 됩니다.

In [ ]:
!cat /content/drive/MyDrive/nmt_fold/data1/*.txt > combined.tsv

인코딩을 UTF_16LE에서 UTF-8로 변경

In [ ]:
!iconv -f UTF-16LE -t UTF-8 /content/drive/MyDrive/nmt_fold/data1/combined.tsv > /content/drive/MyDrive/nmt_fold/data1/combined_utf8.tsv
!mv /content/drive/MyDrive/nmt_fold/data1/combined_utf8.tsv /content/drive/MyDrive/nmt_fold/data1/combined.tsv

In [10]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined*.tsv
!wc -l /content/drive/MyDrive/nmt_fold/data1/combined*.tsv

==> /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv <==
이곳은 공개공지내 인근 건물에서 자체 조성한 공원이지만 흡연자들이 뿜어대는 무분별한 담배연기가 공원외부 보행로까지 확대되어 시민들의 눈살을 찌뿌리게 한 곳이다.	"It is a self-created park created by nearby buildings within the open air space, but reckless cigarette smoke spewed by smokers has expanded to pedestrian paths outside the park, which is frowned upon by the citizens."
미디헴팬티는 기존에 있는 속옷과 다른 것입니까?	Is the midi hem panty different from the original underwear line?
이 가을이 다 가기 전에 창덕궁과 희정당을 찾아 그 속에 담긴 시대적인 아픔을 한번 느껴보는 것도 좋을 것 같다.	"Before this fall is out, it should also be good to visit Changdeokgung Palace and Heejeongdang and feel the pain of the times reflected in the place."
"약 3분의 1은 전혀 반응을 보이지 않으며, 투약 초기 효과를 보이던 환자들도 약 3분의 1은 내성이 생기는지 점차 약발이 듣지 않게 된다."	"About one third do not respond at all, and about one third of the patients who showed the initial effects of the medication become less resistant, which causes the medicine doesn't kick."
"그는 “경제적 포용과 배제를 가르는 관건은 기업의 소유구조”라며, 소유권이

In [7]:
!ls -l /content/drive/MyDrive/nmt_fold/data1/

total 1540803
-rw------- 1 root root  35448312 Aug 13 05:39 01.txt
-rw------- 1 root root  36863298 Aug 13 05:45 02.txt
-rw------- 1 root root  20178746 Aug 13 05:46 03.txt
-rw------- 1 root root 107751918 Aug 13 05:48 04.txt
-rw------- 1 root root  83395820 Aug 13 05:49 05.txt
-rw------- 1 root root 113856170 Aug 13 05:50 06.txt
-rw------- 1 root root  92307742 Aug 13 05:51 07.txt
-rw------- 1 root root  47288970 Aug 13 05:52 08.txt
-rw------- 1 root root  60739970 Aug 13 05:53 09.txt
-rw------- 1 root root  56909742 Aug 13 05:54 10.txt
-rw------- 1 root root 461519736 Aug 13 06:45 combined.shuf.tsv
-rw------- 1 root root 461519736 Aug 13 06:31 combined.tsv


160만 문장을 섞는다.

In [6]:
!shuf /content/drive/MyDrive/nmt_fold/data1/combined.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv

120만개 train,20만개 valid, 202418개 test 나누기

In [9]:
!head -n 1200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv >/content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tsv
!tail -n 402418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv | head -n 200000 > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tsv
!tail -n 202418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv

앞의 column은 한글, 뒤의 column은 영어, 구분해서 파일로 저장

In [11]:
!cut -f1 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko
!cut -f2 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.en
!cut -f1 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko
!cut -f2 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en
!cut -f1 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.ko
!cut -f2 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.en

In [14]:
!wc -l /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*
!head -n 3 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.ko
!head -n 3 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.en

    202418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en
    202418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko
    202418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv
   1200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.en
   1200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko
   1200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tsv
   1602418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv
    200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.en
    200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.ko
    200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tsv
   6409672 total
==> /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko <==
이곳은 공개공지내 인근 건물에서 자체 조성한 공원이지만 흡연자들이 뿜어대는 무분별한 담배연기가 공원외부 보행로까지 확대되어 시민들의 눈살을 찌뿌리게 한 곳이다.
미디헴팬티는 기존에 있는 속옷과 다른 것입니까?
이 가을이 다 가기 전에 창덕궁과 희정당을 찾아 그 속에 담긴 시대적인 아픔을 한번 느껴보는 것도 좋을 것 같다.

==> /content/drive

In [18]:
#!chmod +x /content/drive/MyDrive/nmt_fold/code1/install_mecab_linux.sh
#!bash /content/drive/MyDrive/nmt_fold/code1/install_mecab_linux.sh

# 1) 필수 도구
!apt-get -y update
!apt-get -y install curl

# 2) konlpy 공식 스크립트로 mecab + mecab-ko-dic 설치
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 3) mecabrc의 dicdir 설정(로그에 나온 경로로 맞추기)
!sed -i 's|^dicdir =.*|dicdir = /usr/local/lib/mecab/dic/mecab-ko-dic|' /usr/local/etc/mecabrc

# 4) 파이썬 라이브러리
!pip install konlpy

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,209 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,929 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,270 kB]
Get:13 http://archive.ubuntu.com/ubunt

mecab 설치 확인하기

In [19]:
!mecab -D | head -n 20

filename:	/usr/local/lib/mecab/dic/mecab-ko-dic/sys.dic
version:	102
charset:	UTF-8
type:	0
size:	816283
left size:	3822
right size:	2693



In [21]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko | mecab -O wakati

남부 순환 로 일대 대형 집합 건물 은 주변 아파트 에 비해 노후 화 되 어 낡 은 간판 들 이 주변 환경 과 조화 를 이루 도록 새로운 디자인 의 간판 으로 교체 했 다 . 
우리 나라 는 부품 · 소재 기업 의 역사 가 전통 의 제조업 국가 인 미국 · 유럽 · 일본 등 에 비해 짧 은 만큼 제조 기업 과 장비 기업 의 협력 이 중요 하 다고 전문가 들 은 입 을 모은다 . 
그 는 중의원 7 선 의원 으로 중의원 외무 위원장 까지 지낸 유력 정치인 이 었 지만 평생 세태 와 타협 하 지 않 고 용기 있 게 바른 목소리 를 냈 다 . 
" 그랜드 앰배서더 서울 풀 만 호텔 은 과일 칩 , 육포 , 꽃차 및 소금 2 종 을 담 은 ‘ 맘 스 핸 종합 세트 ’ 를 내놨 는데 ( 7 만 9 천 원 ) 개성 있 는 선물 을 추구 하 는 이 들 에게 호응 이 좋 다 . " 
이것 이 가능 하 면 저 한테 메일 로 연락 하 세요 . 


In [24]:
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko | mecab -O wakati -b 99999  | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko | mecab -O wakati -b 99999  | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.ko
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.ko | mecab -O wakati -b 99999  | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.ko > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.ko


In [27]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.ko
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.ko


▁‘ 얼짱 시대 ’ ▁출신 ▁작가 ▁강혁민 에게 ▁고소 ▁당한 ▁한서희 가 ▁SNS 에 ▁여성 ▁우월주의 ▁성향 의 ▁인터넷 ▁커뮤니티 ▁‘ 워 마드 ( WOMAD ) ’ 를 ▁옹호 하 는 ▁글 을 ▁올려 ▁논란 이 ▁커지 고 ▁있 다 .
▁" 저 는 ▁월미 ▁문화 관 에서 ▁전통 ▁혼례상 을 ▁봤 는데 , ▁상 ▁위 에 ▁살 아 있 는 ▁닭 을 ▁봤 어요 . "
▁‘ 우상 ’ 은 ▁오 는 ▁2 월 ▁7 ∼ 17 일 ▁열리 는 ▁제 69 회 ▁베를린 영화제 ▁비 공식 부문 인 ▁파노라마 ▁섹션 에 ▁초청 됐 다고 ▁이 ▁영화 의 ▁집행 위원회 가 ▁21 일 ( 현지 시간 ) ▁밝혔 다 .
▁대학 ▁캠퍼스 ▁내 ▁위치 한 ▁공연장 은 ▁자유 롭 고 도 ▁여유 ▁있 는 ▁젊은이 들 의 ▁분위기 가 ▁물씬 ▁풍기 는 ▁곳 이 기 도 ▁해 ▁더욱 ▁설렘 을 ▁느끼 게 ▁했 다 .
▁" 암스테르담 ▁음대 ▁역시 ▁OBA 도서관 과 ▁함께 ▁항만 재생 ▁부지 에 ▁입 지 했 는데 , ▁암스테르담 ▁음대 의 ▁사례 를 ▁통해 ▁문화 를 ▁중심 으로 ▁사람 들 이 ▁모이 도록 ▁하 는 ▁항만 재생 의 ▁중요 성 을 ▁다시 ▁한 ▁번 ▁확인 했 다 . "
